In [137]:
#%%
import pandas as pd
import numpy as np

In [138]:
#%%
MONTHS_GEN = {
    1: "января",
    2: "февраля",
    3: "марта",
    4: "апреля",
    5: "мая",
    6: "июня",
    7: "июля",
    8: "августа",
    9: "сентября",
    10: "октября",
    11: "ноября",
    12: "декабря"
}

In [139]:
#%%
data = pd.read_csv('weather_data.csv', sep=',').drop(['text'], axis='columns')

In [140]:
#%%
data.head()

,21-22,8-10,8-9,23-27,16-20,26-29,08-09,9,20-25,10-12,...,14-16,15-17,3-4,26-28,22-24,25,12-14,month,day_start,day_end
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,9,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,10,10
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,9,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,8
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,9,9


In [141]:
#%%
month_list = list(MONTHS_GEN.values())
month_list.append('month')

In [142]:
#%%
data = data.drop(month_list, axis='columns')
data_start = data.drop(['day_end'], axis='columns')

In [143]:
#%%
data_start.head()

,21-22,8-10,8-9,23-27,16-20,26-29,08-09,9,20-25,10-12,...,27,11-13,14-16,15-17,3-4,26-28,22-24,25,12-14,day_start
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,9


In [144]:
#%%
data_start = data_start.drop(data_start[data_start['day_start'] > 31].index)
data_start = data_start.drop(data_start[data_start['day_start'] < 1].index)

In [145]:
#%%
#способ повышения точности-удалить все мешки слов сумма которых больше 1 из датасета
# data_start = data_start.drop(data_start[data_start.iloc[:, 0:118].sum(axis = 1, skipna = True) > 3].index)

In [146]:
#%%

start_x= data_start.drop('day_start', axis=1)
start_y = data_start['day_start'].astype('int')

In [147]:
#%%
start_x.shape, start_y.shape

((11727, 118), (11727,))

In [148]:
#%%
start_y.head()

0     9
1    10
2     9
3     8
4     9
Name: day_start, dtype: int32

In [149]:
#%%
start_y.describe()

count    11727.000000
mean        15.392428
std          8.830100
min          1.000000
25%          8.000000
50%         15.000000
75%         23.000000
max         31.000000
Name: day_start, dtype: float64

In [150]:
#%%
from sklearn.tree import DecisionTreeClassifier
first_tree = DecisionTreeClassifier(random_state=17)

In [151]:
#%%
from sklearn.model_selection import train_test_split
start_x_train, start_x_valid, start_y_train, start_y_valid = train_test_split(start_x, start_y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [152]:
#%%
start_x_train.shape, start_x_valid.shape

((8208, 118), (3519, 118))

In [153]:
#%%
from sklearn.model_selection import  cross_val_score
cross_val_score(first_tree, start_x_train, start_y_train, cv=5)

array([0.8727162 , 0.86723508, 0.86419001, 0.87507617, 0.87446679])

In [154]:
#%%
from sklearn.neighbors import KNeighborsClassifier

In [155]:
#%%
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, start_x_train, start_y_train, cv=5))

0.8717110493067045

In [156]:
#%%
%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=47, random_state=17,
    n_jobs=-1, max_features=2,
    )
rf.fit(start_x_train, start_y_train)

Wall time: 0 ns


RandomForestClassifier(max_features=2, n_estimators=47, n_jobs=-1,
                       random_state=17)

In [157]:
#%%
%time
rf_valid_pred = rf.predict(start_x_valid)

Wall time: 0 ns


In [158]:
#%%
from sklearn.metrics import accuracy_score
accuracy_score(start_y_valid, rf_valid_pred)

0.8883205456095482

In [159]:
#%%
# from sklearn.metrics import confusion_matrix
# confusion_matrix(start_y_valid, rf_valid_pred)

In [160]:
#%%
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [161]:
#%%
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [162]:
#%%
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

In [163]:
#%%
results = cross_val_score(rfc, start_x, start_y, cv=skf)

In [164]:
#%%
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

CV accuracy score: 88.25%


In [165]:
#%%
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [166]:
#%%
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = range(90,100) 

In [167]:
#%%
# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

In [168]:
#%%
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} trees".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.31% with 91 trees


In [169]:
#%%
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_depth_grid = range(80,120)

In [170]:
#%%
# Обучаем на тренировочном датасете
for max_depth in max_depth_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, oob_score=True, max_depth=max_depth)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_depth".format(max(test_acc.mean(axis=1))*100, 
                                                        max_depth_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.39% with 94 max_depth


In [171]:
#%%
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
min_samples_leaf_grid = [1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

In [172]:
#%%
# Обучаем на тренировочном датасете
for min_samples_leaf in min_samples_leaf_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, 
                                 oob_score=True, min_samples_leaf=min_samples_leaf, max_depth=94)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} min_samples_leaf".format(max(test_acc.mean(axis=1))*100, 
                                                                min_samples_leaf_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.39% with 1 min_samples_leaf


In [173]:
#%%
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_features_grid = [2, 4, 6, 8, 10, 12, 14, 16]

# Обучаем на тренировочном датасете
for max_features in max_features_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, 
                                 oob_score=True, max_features=max_features)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_features".format(max(test_acc.mean(axis=1))*100, 
                                                        max_features_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.31% with 10 max_features
